# EDArtifact Detection Pipeline

This notebook contains the steps needed for running the Electrodermal Activity artifacts (EDArtifact) detection pipeline explained in the following paper: 

[1] Gashi, Shkurta, Elena Di Lascio, Bianca Stancu, Vedant Das Swain, Varun Mishra, Martin Gjoreski, and Silvia Santini. "Detection of Artifacts in Ambulatory Electrodermal Activity Data." In: Proceedings of the ACM on Interactive, Mobile, Wearable and Ubiquitous Technologies (PACM IMWUT) Vol. 4, Issue 2, June 2020. Pages: 1-31.

If you use this script or the model to identify artifacts in your data please make sure to cite our paper [1], which is available at https://dl.acm.org/doi/10.1145/3397316.




## Import the required packages

In order to be able to identify artifacts in EDA signals the following packages are needed:
- *MinMaxScaler*: for normalizing the features before providing them as input to the model.
- *linregress*: to calculate the linear regression for each 5 second segment.
- *signal* and *butter*: to apply the Butterworth filter on the EDA signal.
- *pickle*: to load the trained model for detecting artifacts.  
- *cvxEDA*: to decompos the EDA signal into the phasic and tonic component.
- *pywt*: to compute the wavelets. 

In [3]:
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import linregress
from scipy import signal
from scipy.signal import butter
from itertools import chain
import pandas as pd
import numpy as np
import pickle
import cvxEDA
import pywt
import plotly.graph_objects as go
import plotly.express as px
import xgboost

## 1. Preprocess EDA signals

We first apply a common procedure for pre-processing EDA data similar to the one described in the literature. Preprocessing of EDA signals includes three main steps:
- Filtering 
- Decomposition
- Wavelets calculation

The decomposition step implements the cvxEDA algorithm for the analysis of electrodermal
activity (EDA) using methods of convex optimization, described in:

[2] A Greco, G Valenza, A Lanata, EP Scilingo, and L Citi
"cvxEDA: a Convex Optimization Approach to Electrodermal Activity Processing"
IEEE Transactions on Biomedical Engineering, 2015
DOI: 10.1109/TBME.2015.2474131. 

If you use this part of the code, please cite paper [2]. For more details for the decomposition of EDA signals please refer to https://github.com/lciti/cvxEDA/blob/master/test/test_simul.m.

## 2. Segment the EDA signals

Given that a SCR response lasts between 1 to 5 seconds, we then divide the EDA traces in 5 second non-overlapping segments as a common procedure in shape artifacts detection.

## 3. Compute the features

We extract features from the 5-second non-overlapping segments of EDA that could appropriately characterize the shape artifacts. The three feature groups involve:
- **Statistical features**: We extract the common statistical features including features such as the minimum, maximum, mean, median, standard deviation, dynamic rannge, slope of the signal and mean and standard deviation of the first and second derivative.
- **Peaks features**: We extract 6 SCR features from the phasic component of EDA, including the number of peaks, peaks’ rise time, half-recovery time, amplitude, width and area under the curve.
- **Wavelets features**: We extract wavelet features that might be indicative of sudden changes in EDA. We extract the coefficients of a Discrete Wavelet Transform (DWT) with the Haar wavelet applied in the EDA signals at three different time scales: 4 Hz, 2 Hz and 1 Hz.

## 3.1 Compute *statistical* and *wavelets* features

## 3.2 Compute *peaks* features 

This part of the code is adapted from the EDA peak detection script available at: https://github.com/MITMediaLabAffectiveComputing/eda-explorer. If you use this part of the code, please cite the following paper:

Taylor, Sara, Natasha Jaques, Weixuan Chen, Szymon Fedor, Akane Sano, and Rosalind Picard. "Automatic identification of artifacts in electrodermal activity data." In 2015 37th Annual International Conference of the IEEE Engineering in Medicine and Biology Society (EMBC), pp. 1934-1937. IEEE, 2015.

## 4. Remove flat responses 

We then discard the EDA segments with a slope in range [-0.002, +0.002] because we consider them as flat responses, which do not contain for instance skin conductance responses (SCRs) and hence do not contain shape artifacts.

## 5. Predict EDA artifacts using our trained model

We provide the computed features as an input to the trained model to find the artifacts in the EDA data.

## 6. Merge identified artifacts to the whole signal

## Definition of EDArtifact Detection pipeline 

## Run the EDArtifact Detection pipeline

In [5]:
from support import compute_eda_artifacts
"""
    TODO: provide the path to your EDA file
    
    Note that the EDA file should contain two columns: EDA and Time. 
    The format of the Time column should be as following YYYY-mm-dd HH:MM:SS.milliseconds
"""
file_path = "/Users/leonardoalchieri/Datasets/USILaughs/participants/s037/left/EDA.csv"
compute_eda_artifacts(
    file_path,
    show_database=True,
    convert_dataframe=True,
    output_path="test.csv",
    header=[0, 1],
    window_size=4,
)


                                 Time       EDA
0           2018-06-14 15:13:16+02:00  0.000000
1    2018-06-14 15:13:16.250000+02:00  0.375272
2    2018-06-14 15:13:16.500000+02:00  0.457242
3    2018-06-14 15:13:16.750000+02:00  0.467489
4           2018-06-14 15:13:17+02:00  0.468770
...                               ...       ...
5227 2018-06-14 15:35:02.750000+02:00  0.017931
5228        2018-06-14 15:35:03+02:00  0.021773
5229 2018-06-14 15:35:03.250000+02:00  0.019212
5230 2018-06-14 15:35:03.500000+02:00  0.020493
5231 2018-06-14 15:35:03.750000+02:00  0.021773

[5232 rows x 2 columns]
     pcost       dcost       gap    pres   dres
 0: -1.4841e+03 -1.4798e+03  5e+03  7e+01  7e-02
 1: -1.4800e+03 -1.7368e+03  3e+02  4e+00  3e-03
 2: -1.4802e+03 -1.5352e+03  6e+01  7e-01  6e-04
 3: -1.4807e+03 -1.4878e+03  7e+00  4e-02  4e-05
 4: -1.4839e+03 -1.4847e+03  8e-01  4e-04  4e-07
 5: -1.4842e+03 -1.4847e+03  5e-01  2e-04  2e-07
 6: -1.4846e+03 -1.4847e+03  9e-02  3e-05  2e-08
 7: -1.4

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 20 and the array at index 1 has size 16

In [3]:
'''
    TODO: provide the path to your EDA file
    
    Note that the EDA file should contain two columns: EDA and Time. 
    The format of the Time column should be as following YYYY-mm-dd HH:MM:SS.milliseconds
''' 
file_path = "Data/EDA_Table_Day.csv"
compute_eda_artifacts(file_path, show_database=True, header=0, window_size=4)

NameError: name 'compute_eda_artifacts' is not defined

## Plot the EDA signal with detected artifacts

In [4]:
import pandas as pd
data = pd.read_csv("test.csv")
data_sa = data[data.Artifact == 1]
starting_timestamps = data_sa.iloc[::20, :].Time.values
starting_indices = list(data_sa.iloc[::20, :].index.values)
ending_indices = [x + 19 for x in starting_indices]
ending_timestamps = data[data.index.isin(ending_indices)].Time.values


fig = px.line(data, x='Time', y='EDA', title='EDA with shape artifacts highlighted' )

fig.update_xaxes(rangeslider_visible=True)

fig.update_layout(
    shapes=[
        dict(
            type="rect",
            xref="x",
            yref="paper",
            x0=i,
            y0=0,
            x1=j,
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.9,
            layer="below",
            line_width=0,
        ) for i,j in zip(starting_timestamps, ending_timestamps)
    ]
)

fig.show()